# Text Classification

# Bayes Classifier

Bayes classifiers fall under the class of **generative classifiers**. Generative classifiers attempt to learn the generation process of a dataset, usually by making some assumptions about the process that generates the data. Then such classifiers use the learned model to make a prediction or classify the unseen data. A simple example is a Naïve Bayes Classifier.

### Naïve Bayes classifier
Consider a dataset $\left\{X^{(i)}, Y^{(i)}\right\}_{i=1}^{m}$. Each $X^{(i)}$ is an $n-$dimensional vector of input features. Let $Y^{(i)} \in \{0,1\}$ denote the class to which $X^{(i)}$ belongs (this can be easily extended to multi-class problems as well). A good classifier has to accurately predict the probability that any given input $X$ falls in class $1$ which is $ P(Y=1 | X)$. 

Recall Bayes theorem,

\begin{align}
P(Y|X) &= \frac{P(X|Y)P(Y)}{P(X)} \\
       &= \frac{P(X_1, X_2, \dots, X_n | Y)P(Y)}{P(X_1, X_2, \dots, X_n)}\\
\end{align}

**We use the assumption that features are independent of each other. That is one particular feature does not affect any other feature. Of course these assumptions of independence are rarely true, which is why the model is referred as the "Naïve Bayes" model. However, in practice, Naïve Bayes models have performed surprisingly well even on complex tasks, where it is clear that the strong independence assumptions are false.**

The independence assumption reduces the conditional probability expression to
\begin{align}
P(Y|X) &= \frac{P(X_1 | Y)P(X_2 | Y) \dots P(X_n | Y)P(Y)}{P(X_1)P(X_2)\dots P(X_n)}\\
\end{align}

The terms $P(X_i|Y)$ and $P(X_i)$ can be easily estimated/learned from the dataset. Hence, the value of $P(Y|X)$ can be found for each value of $Y$. Finally, the class to which $X$ belongs is estimated as $arg\max_{Y}P(Y|X)$. Moreover since $X$ is independent of $Y$, it is only required to find $arg\max_{Y}P(X|Y)P(Y).$ For better understanding with an example refer [this](https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c) article.


### Problem statement and Dataset
In this problem, you would implement, train and test a Naïve Bayes model to learn to classify sentiment (positive/negative) of a given text. The training data is in `all_sentiment_shuffled.txt` file.  You can use the function given below to read the dataset

In [1]:
#nltk, numpy, warnings, libraries were used here.
import nltk
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#This code reads the corpus file and separates them into text and label.
def read_corpus(corpus_file):
    """ This function reads the file in the location specified by string 
    `corpus_file` and returns a list of tuples (list of words in text, label)
    """
    out = []
    with open(corpus_file,encoding="utf8") as f:
        for line in f:
            tokens = line.strip().split()
            out.append((tokens[3:], tokens[1]))
    return out

In [3]:
#function is called and spliied into text and label.
corpus = read_corpus('./all_sentiment_shuffled.txt')
print("Example:\n", " Text: ", corpus[1][0], "\n  Label: ", corpus[1][1])
print("Total number of documents =", len(corpus))

Example:
  Text:  ['i', 'was', 'misled', 'and', 'thought', 'i', 'was', 'buying', 'the', 'entire', 'cd', 'and', 'it', 'contains', 'one', 'song'] 
  Label:  neg
Total number of documents = 11914


### Preprocessing a text document
We can guess that not all the words in a document will be helpful in classification. The words such as "a", "the", "is", etc appear in all the documents randomly and can be neglected or removed. Also a same word can be written in different tenses while conveying the same mood (example "rot"/"rotten"). Hence the documents need to be preprocessed before using them for training the classifier.

 Libraries such as `gensim`, `nltk` contain functions for doing these preprocessing steps, and you are welcome to use such functions in your code. Formally, these are the preprocessings to be done to the input text to make them simpler and which can improve the performance of your model as well.
* **Tokenization**: 
    1.   Split the text into sentences and the sentences into words
    2.   Lowercase the words and remove punctuation
* Remove all **stopwords** (stopwords are commonly used word such as "the", "a", "an", "in")
* Remove all words that have fewer than 3 characters.
* **Lemmatize** the document (words in third person are changed to first person, and verbs in past and future tenses are changed into present).


 Removes all the punctuations present in the document

In [4]:
#Following was imported from nltk for preprocessing.
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer

In [5]:
# Removes all the punctuations present in the document
def remove_punctuation(doc,n):
    tokenizer = RegexpTokenizer(r'\w+')
    punc_removed=tokenizer.tokenize(" ".join(doc[n][0]))
    
    return punc_removed

 Removes words like 'if', 'he', 'she', 'the', etc which never belongs to any topic and Remove all words that have fewer than 3 characters.

In [6]:
# Removes words like 'if', 'he', 'she', 'the', etc which never belongs to any topic and Remove all words that have fewer than 3 characters.
def remove_stopwords(doc):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(" ".join(doc))
   
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w.lower()) 
            
    filtered_sentence=[l for l in filtered_sentence if len(l)>=3]
    
    return filtered_sentence

 lemmatizer is a transformers which transforms the word to its singular, present-tense form

In [7]:
# lemmatizer is a transformers which transforms the word to its singular, present-tense form
def lemmatize(doc):
    lemmatizer = WordNetLemmatizer()
    lemmatizer_sentence=[lemmatizer.lemmatize(w)for w in doc]
    
    return lemmatizer_sentence
#nltk.download('wordnet') 

Function to preprocess a single document

In [8]:
#All the functions are combined together into a single preprocessing document.
def preprocess(doc,n):
    processed_doc = remove_punctuation(doc,n)
    processed_doc = remove_stopwords(processed_doc)
    processed_doc = lemmatize(processed_doc)
    return processed_doc

### Implementation of Naïve Bayes 

You can refer the Naïve Bayes section in [this](https://web.stanford.edu/~jurafsky/slp3/slides/7_NB.pdf) slides (slide #32 has a simple pseudo code) to get a hint about implementation of Naïve Bayes for text classification. Then complete the following functions `train_nb` and `classify_nb`.

NOTE: If you multiply many small probabilities you may run into problems with numeric precision: the probability becomes zero. To handle this problem, it is recommended that you compute the logarithms of the probabilities instead of the probabilities.

### Train-test split
After reading the dataset, you must split the dataset into training ($80\%$) and test data ($20\%$). Use training data to train the Naïve Bayes classifier and use test data to check the accuracy.

In [9]:
#creating testing and training datasets.
import random
#80 percent of the data is used for training.
test=round(0.8*len(corpus))
rang=len(corpus)
training_data = random.sample(range(0,rang), test)
 
#20 percent of the data is used for testing.    
testing_data=[]
for i in range(0,rang):
    if i in training_data:
        continue
    else:
        testing_data.append(i)        

In [10]:
#finding the probability of positive,negative statement,words.
#This code helps in finding the probability of positive  and negative statements and classify words accordingly.
positive=0
negative=0
positive_words=[]
negative_words=[]
for train_data in training_data:
    if corpus[train_data][1]=='neg':
        negative += 1
        modin=preprocess(corpus,train_data)
        for i in modin:
            negative_words.append(i)
        
    else:
        positive +=1 
        modip=preprocess(corpus,train_data)
        for j in modip:
            positive_words.append(j)
        

#This is the probability of postive and negative statements.
Prob_pos=positive/len(training_data)
Prob_neg=negative/len(training_data)

In [11]:
#This is the probability of postive statements.
Prob_pos

0.5049837372783549

In [12]:
#This is the probability of negative statements.
Prob_neg

0.4950162627216452

In [13]:
#laplace smoothing technique is considered here and with alpha=0.
def lap_smo(Nxw,Nw,d):
    return (Nxw+1)/(Nw+d)

In [14]:
#Here d signifies the total number of unique words both in postive and negative words list.
d=len(set(positive_words))+len(set(negative_words))

In [15]:
#Here d signifies the total number of unique words both in postive and negative words list.
d=len(set(positive_words))+len(set(negative_words))
d

51309

In [16]:
#positive words count in dictionary.
postive_words_set=set(positive_words)
positive_dict={}
for dict_num1 in postive_words_set:
    count_num1=positive_words.count(dict_num1)
    positive_dict.update({dict_num1: count_num1})

In [17]:
#negative words count in dictionary.
negative_words_set=set(negative_words)
negative_dict={}
for dict_num2 in negative_words_set:
    count_num2=negative_words.count(dict_num2)
    negative_dict.update({dict_num2: count_num2})

In [18]:
#This code helps in classifying a given statement into positive and negative label.
npo=len(positive_words)
nne=len(negative_words)

result=[]
#The the code is runing for only 100 testing data_set i have not included all because of the large computation time it takes.
#we test for all testing data by removing [0:100].
for test_data in testing_data:
    
    prob_p_x=1
    prob_n_x=1
    
    mod=preprocess(corpus,test_data)
    
    for i in mod:
        if i in postive_words_set:
            tmp_p=positive_dict[i]
            tmp_p=lap_smo(tmp_p,npo,d)
            prob_p_x *= np.log10(tmp_p)
        else:
            tm_p=0
            tmp_p=lap_smo(tmp_p,npo,d)
            prob_p_x *= np.log10(tmp_p)
            
    
    for j in mod:
        if j in negative_words_set:
            tmp_n=negative_dict[j]
            tmp_n=lap_smo(tmp_n,nne,d)
            prob_n_x *= np.log10(tmp_n)
        else:
            tmp_n=0
            tmp_n=lap_smo(tmp_n,nne,d)
            prob_n_x *= np.log10(tmp_n)
            
      
    prob_p_x=abs(prob_p_x*np.log10(Prob_pos))
    prob_n_x=abs(prob_n_x*np.log10(Prob_neg))
    
    if prob_p_x < prob_n_x:
        result.append(1)
  
    else:
        result.append(-1)

In [19]:
#This gives the total number of testing statements.
len(result)

2383

In [20]:
#This code helps in finding accuracy of the above model.
compare=[]
#The the code is runing for only 100 testing data_set i have not included all because of the large computation time it takes.
#we test for all testing data by removing [0:100].
for test_data in testing_data:
    if corpus[test_data][1]=='pos':
        compare.append(1)
    else:
        compare.append(-1)
        
accu=np.array(result)+np.array(compare)

accuracy=list(accu).count(0)/len(result)
accuracy=(1-accuracy)*100

In [21]:
#This is the accuracy value.
accuracy

80.86445656735208

# Observation and Result:

In [22]:
#This is the final accuracy result.
print('The model above constructed is {} % accurate'.format(accuracy))

The model above constructed is 80.86445656735208 % accurate
